In [1]:
import csv
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)
es = EarlyStopping(monitor='val_accuracy',mode='max')

In [2]:
df = pd.read_csv('C:/git_project/natural/data/okt/X_train.csv')
df

,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,44
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,72,528,32,408,699
1,0,0,0,0,0,0,0,0,0,0,...,507,45,674,2,101,1360,36,1003,720,35
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,19,577,4,21,8,234,3
3,0,0,0,0,0,0,0,0,0,0,...,0,6396,125,5999,253,70,12,14,49,3566
4,0,0,0,0,0,0,0,0,0,0,...,1122,257,3,10090,11825,7,1116,1,1404,257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155863,0,0,0,0,0,1087,4173,5,7939,8,...,15,40,73,182,39,444,66,10,266,596
155864,0,0,0,0,0,0,0,0,0,0,...,146,6,819,1907,659,15,3,692,4854,35
155865,0,0,0,0,0,0,0,0,0,0,...,105,191,105,191,95,45,61,1116,865,636
155866,0,0,0,0,0,0,0,0,0,0,...,5,285,1116,1610,41,5,1461,1116,93,3


In [3]:
X_train_data = pd.read_csv('C:/git_project/natural/data/okt/X_train.csv')   # 데이터 불러오기
X_test_data = pd.read_csv('C:/git_project/natural/data/okt/X_test.csv')
y_train_data = pd.read_csv('C:/git_project/natural/data/okt/y_train.csv')   # 데이터 불러오기
y_test_data = pd.read_csv('C:/git_project/natural/data/okt/y_test.csv')

In [4]:
X_train_data.dropna(how='any', inplace=True)
X_test_data.dropna(how='any', inplace=True)
y_train_data.dropna(how='any', inplace=True)
y_test_data.dropna(how='any', inplace=True)

In [5]:
train_X = np.array(X_train_data)
test_X = np.array(X_test_data)
train_X, test_X

(array([[   0,    0,    0, ...,   32,  408,  699],
        [   0,    0,    0, ..., 1003,  720,   35],
        [   0,    0,    0, ...,    8,  234,    3],
        ...,
        [   0,    0,    0, ..., 1116,  865,  636],
        [   0,    0,    0, ..., 1116,   93,    3],
        [   0,    0,    0, ...,   95,  154,    4]], dtype=int64),
 array([[   0,    0,    0, ...,    0,  815,  119],
        [   0,    0,    0, ...,   64,  745,   34],
        [   0,    0,    0, ...,   86,  170,  295],
        ...,
        [   0,    0,    0, ...,  180, 3045,   40],
        [   0,    0,    0, ...,   11,  255,  151],
        [   0,    0,    0, ...,  143,   59,   46]], dtype=int64))

In [6]:
train_y = np.array(y_train_data)
test_y = np.array(y_test_data)
train_y, test_y

(array([[0],
        [1],
        [0],
        ...,
        [1],
        [1],
        [1]], dtype=int64),
 array([[1],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]], dtype=int64))

# Model Building

In [7]:
vocab_size = 19395

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 32, mask_zero=True))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Bidirectional(keras.layers.GRU(64)))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(16, activation='tanh'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          620640    
                                                                 
 dropout (Dropout)           (None, None, 32)          0         
                                                                 
 bidirectional (Bidirectiona  (None, 128)              37632     
 l)                                                              
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 16)                2064      
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                        

In [8]:
model.compile(optimizer=keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Training

In [9]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_data, y_train_data))
val_dataset = tf.data.Dataset.from_tensor_slices((X_test_data, y_test_data))

train_dataset = train_dataset.shuffle(buffer_size=1024, reshuffle_each_iteration=True).batch(128)
val_dataset = val_dataset.batch(1000)

train_dataset = train_dataset.prefetch(4)

In [ ]:
history = model.fit(train_dataset, epochs=10, validation_data=val_dataset, verbose=1,callbacks=[mc])

Epoch 1/20
1218/1218 [==============================] - 128s 100ms/step - loss: 0.4158 - accuracy: 0.8118 - val_loss: 0.4884 - val_accuracy: 0.7608
Epoch 2/20
 967/1218 [======================>.......] - ETA: 22s - loss: 0.3532 - accuracy: 0.8527

In [ ]:
# print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test_data, y_test_data)[1]))

In [ ]:
# sentiment_predict('나는')

In [ ]:
# def sentiment_predict(new_sentence):
#   # 알파벳과 숫자를 제외하고 모두 제거 및 알파벳 소문자화
#     new_sentence = re.sub('[^0-9a-zA-Z ]', '', new_sentence).lower()

#   # 정수 인코딩
#     encoded = []
#     for word in history.split():
#         try :
#           # 단어 집합의 크기를 10,000으로 제한.
#             if word_to_index[word] <= 10000:
#                 encoded.append(word_to_index[word]+3)
#             else:
#             # 10,000 이상의 숫자는 <unk> 토큰으로 변환.
#                 encoded.append(2)
#             # 단어 집합에 없는 단어는 <unk> 토큰으로 변환.
#         except KeyError:
#             encoded.append(2)

#     pad_sequence = pad_sequences([encoded], maxlen=max_len) # 패딩
#     score = float(loaded_model.predict(pad_sequence)) # 예측

#     if(score > 0.5):
#         print("{:.2f}% 확률로 긍정 리뷰입니다.".format(score * 100))
#     else:
#         print("{:.2f}% 확률로 부정 리뷰입니다.".format((1 - score) * 100))

In [ ]:
# sentiment_predict('행복합니다')